# Plot bottom age + salinity anomonalies for WIND+, WIND-

In [21]:
%matplotlib inline

import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import matplotlib.colors as col
import cartopy.crs as ccrs
import cartopy.mpl.ticker as cticker
import matplotlib.path as mpath
import cosima_cookbook as cc
import cmocean.cm as cmocean
import cmocean.cm as cm
from collections import OrderedDict
import matplotlib.gridspec as gridspec
import matplotlib.patches as mpatches

import warnings
warnings.filterwarnings('ignore')
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

In [22]:
from dask.distributed import Client
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/36589/status,
Dashboard: /proxy/36589/status,Workers: 7
Total threads: 28,Total memory: 125.20 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:38763,Workers: 7
Dashboard: /proxy/36589/status,Total threads: 28
Started: Just now,Total memory: 125.20 GiB
Comm: tcp://127.0.0.1:43729,Total threads: 4
Dashboard: /proxy/41223/status,Memory: 17.89 GiB
Nanny: tcp://127.0.0.1:34907,


In [23]:
# database for control simulation:
master_session = cc.database.create_session('/g/data/ik11/databases/cosima_master.db')
control = '01deg_jra55v13_ryf9091'

# database for perturbations:
easterlies_session = cc.database.create_session('/g/data/v45/akm157/model_data/access-om2/01deg_jra55v13_ryf9091_easterlies_up10/easterlies.db')
# 10% increase in easterly wind speed:
easterlies_up = '01deg_jra55v13_ryf9091_easterlies_up10'
# 10% decrease in easterly wind speed:
easterlies_down = '01deg_jra55v13_ryf9091_easterlies_down10'
# 10% increase in easterly wind speed, meridional component only:
easterlies_up_mer = '01deg_jra55v13_ryf9091_easterlies_up10_meridional'
# 10% increase in easterly wind speed, zonal component only:
easterlies_up_zon = '01deg_jra55v13_ryf9091_easterlies_up10_zonal'

# plot anomalies for 5 years, years 5-10 of simulation:
start_time = '2159-01-01' 
start_time_5 = '2155-01-01'
end_time   = '2159-12-31'

lat_slice  = slice(-80,-59)

In [24]:
# topography data for plotting:
ht = cc.querying.getvar(control,'ht' , master_session,n=1)
ht = ht.sel(yt_ocean=lat_slice)
ht = ht.load()
land_mask = np.squeeze(ht.values)*0
land_mask[np.isnan(land_mask)] = 1
land_mask = np.where(land_mask==1,land_mask,np.nan)
land_mask_masked = np.ma.masked_where((land_mask==0),land_mask)
# make land go all the way to -90S:
land_mask_lat = ht.yt_ocean.values
land_mask_lat[0] = -90

fontsize=13

## Import bottom age and salinity

Age

In [25]:
age_con = cc.querying.getvar(control,'age_global' , master_session,frequency='1 monthly',start_time=start_time, end_time=end_time)
age_con = age_con.sel(yt_ocean=lat_slice).sel(time=slice(start_time,end_time)).mean('time')                             

age_up = cc.querying.getvar(easterlies_up, 'age_global', easterlies_session, frequency='1 monthly', start_time=start_time, end_time=end_time)
age_up = age_up.sel(yt_ocean=lat_slice).sel(time=slice(start_time,end_time)).mean('time')                             

age_down = cc.querying.getvar(easterlies_down,'age_global', easterlies_session, frequency='1 monthly', start_time=start_time, end_time=end_time)
age_down = age_down.sel(yt_ocean=lat_slice).sel(time=slice(start_time,end_time)).mean('time')                             

Exception during reset or similar
Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.04/lib/python3.9/site-packages/sqlalchemy/pool/base.py", line 697, in _finalize_fairy
    fairy._reset(pool)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.04/lib/python3.9/site-packages/sqlalchemy/pool/base.py", line 893, in _reset
    pool._dialect.do_rollback(self)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.04/lib/python3.9/site-packages/sqlalchemy/engine/default.py", line 558, in do_rollback
    dbapi_connection.rollback()
sqlite3.ProgrammingError: SQLite objects created in a thread can only be used in that same thread. The object was created in thread id 22659627730752 and this is thread id 22654351820544.
Exception closing connection <sqlite3.Connection object at 0x149b2d1da8a0>
Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.04/lib/python3.9/site-packages/sqlalchemy/pool/b

In [26]:
# select out bottom values:
depth_array = age_con * 0 + age_con.st_ocean
max_depth = depth_array.max(dim = 'st_ocean', skipna= True)

In [27]:
bottom_age_con = age_con.where(depth_array.st_ocean >= max_depth)
bottom_age_con = bottom_age_con.sum(dim = 'st_ocean')
bottom_age_con = bottom_age_con.load()

/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.04/lib/python3.9/site-packages/dask/array/reductions.py:608: RuntimeWarning: All-NaN slice encountered
  return np.nanmax(x_chunk, axis=axis, keepdims=keepdims)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.04/lib/python3.9/site-packages/dask/array/reductions.py:608: RuntimeWarning: All-NaN slice encountered
  return np.nanmax(x_chunk, axis=axis, keepdims=keepdims)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.04/lib/python3.9/site-packages/dask/array/reductions.py:608: RuntimeWarning: All-NaN slice encountered
  return np.nanmax(x_chunk, axis=axis, keepdims=keepdims)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.04/lib/python3.9/site-packages/dask/array/reductions.py:608: RuntimeWarning: All-NaN slice encountered
  return np.nanmax(x_chunk, axis=axis, keepdims=keepdims)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.04/lib/python3.9/site-packages/dask/array/reductions.py:608: RuntimeWarning: All-NaN slice 

In [28]:
bottom_age_up = age_up.where(depth_array.st_ocean >= max_depth)
bottom_age_up = bottom_age_up.sum(dim = 'st_ocean')
bottom_age_up = bottom_age_up.load()

In [29]:
bottom_age_down = age_down.where(depth_array.st_ocean >= max_depth)
bottom_age_down = bottom_age_down.sum(dim = 'st_ocean')
bottom_age_down = bottom_age_down.load()

Salinity

In [30]:
salt_con = cc.querying.getvar(control,'salt' , master_session,frequency='1 monthly',start_time=start_time_5, end_time=end_time)
salt_con = salt_con.sel(yt_ocean=lat_slice).sel(time=slice(start_time_5,end_time)).mean('time')                             

salt_up = cc.querying.getvar(easterlies_up, 'salt', easterlies_session, frequency='1 monthly', start_time=start_time_5, end_time=end_time)
salt_up = salt_up.sel(yt_ocean=lat_slice).sel(time=slice(start_time_5,end_time)).mean('time')                             

salt_down = cc.querying.getvar(easterlies_down,'salt', easterlies_session, frequency='1 monthly', start_time=start_time_5, end_time=end_time)
salt_down = salt_down.sel(yt_ocean=lat_slice).sel(time=slice(start_time_5,end_time)).mean('time')    

In [31]:
bottom_salt_con = salt_con.where(depth_array.st_ocean >= max_depth)
bottom_salt_con = bottom_salt_con.sum(dim = 'st_ocean')
bottom_salt_con = bottom_salt_con.load()

/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.04/lib/python3.9/site-packages/dask/array/reductions.py:608: RuntimeWarning: All-NaN slice encountered
  return np.nanmax(x_chunk, axis=axis, keepdims=keepdims)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.04/lib/python3.9/site-packages/dask/array/reductions.py:608: RuntimeWarning: All-NaN slice encountered
  return np.nanmax(x_chunk, axis=axis, keepdims=keepdims)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.04/lib/python3.9/site-packages/dask/array/reductions.py:608: RuntimeWarning: All-NaN slice encountered
  return np.nanmax(x_chunk, axis=axis, keepdims=keepdims)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.04/lib/python3.9/site-packages/dask/array/reductions.py:608: RuntimeWarning: All-NaN slice encountered
  return np.nanmax(x_chunk, axis=axis, keepdims=keepdims)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.04/lib/python3.9/site-packages/dask/array/reductions.py:608: RuntimeWarning: All-NaN slice 

In [32]:
bottom_salt_up = salt_up.where(depth_array.st_ocean >= max_depth)
bottom_salt_up = bottom_salt_up.sum(dim = 'st_ocean')
bottom_salt_up = bottom_salt_up.load()

/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.04/lib/python3.9/site-packages/dask/array/reductions.py:608: RuntimeWarning: All-NaN slice encountered
  return np.nanmax(x_chunk, axis=axis, keepdims=keepdims)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.04/lib/python3.9/site-packages/dask/array/reductions.py:608: RuntimeWarning: All-NaN slice encountered
  return np.nanmax(x_chunk, axis=axis, keepdims=keepdims)


In [33]:
bottom_salt_down = salt_down.where(depth_array.st_ocean >= max_depth)
bottom_salt_down = bottom_salt_down.sum(dim = 'st_ocean')
bottom_salt_down = bottom_salt_down.load()

## Plotting

In [34]:
# Figure path
#fig_path = '/g/data/x77/wgh581/Figures/Easterlies-Collab/'
fig_path = '/g/data/v45/akm157/figures/easterlies/paper_figures/'
# Panel labels
panel_name = ['a)', 'b)', 'c)', 'd)']
# Font size
plt.rcParams['font.size'] = 18
# Axes
plt.rcParams['axes.facecolor']  = 'white'
plt.rcParams['xtick.labelsize'] = 18
plt.rcParams['ytick.labelsize'] = 18
# Saving parameters
plt.rcParams['savefig.dpi']  = 150
plt.rcParams['savefig.bbox'] = 'tight'

Isobath

In [35]:
contour_data  = np.load('/g/data/v45/akm157/model_data/access-om2/Antarctic_slope_contour_1000m.npz')
contour_mask_numbered = contour_data['contour_mask_numbered']
contour_1000m = np.where(contour_mask_numbered == 0, contour_mask_numbered, 1000)
contour_xt_ocean = contour_data['xt_ocean']
contour_yt_ocean = contour_data['yt_ocean']

In [44]:
# DSW regions:
poly1 = mpatches.Polygon(np.transpose(np.array([[161.5-360,170-360,170-360,161.5-360,161.5-360],[-79,-79,-72,-72,-79]])), closed = True, ec = 'darkgoldenrod', 
                        fill = False, lw = 2, fc = 'goldenrod', alpha = 0.8, zorder = 2)
poly2 = mpatches.Polygon(np.transpose(np.array([[58,73,73,58,58],[-69,-69,-67,-67,-69]])), closed = True, ec = 'darkgoldenrod', 
                        fill = False, lw = 2, fc = 'goldenrod', alpha = 0.8, zorder = 2)
poly3 = mpatches.Polygon(np.transpose(np.array([[138-360,148-360,148-360,138-360,138-360],[-69,-69,-66.2,-66.2,-69]])), closed = True, ec = 'darkgoldenrod', 
                        fill = False, lw = 2, fc = 'goldenrod', alpha = 0.8, zorder = 2)
poly4 = mpatches.Polygon(np.transpose(np.array([[297-360,305-360,305-360,320-360,320-360,305-360,305-360,297-360,297-360],[-70,-70,-75,-75,-79,-79,-77,-77,-70]])), closed = True, ec = 'darkgoldenrod', 
                        fill = False, lw = 2, fc = 'goldenrod', alpha = 0.8, zorder = 2)

In [ ]:
fig = plt.figure(figsize=(15, 17))
# Panel title
title_name = ['WIND+', 'WIND-', 'WIND+', 'WIND-']
# Subplots grid
gs  = gridspec.GridSpec(4, 2, height_ratios=[1, 1, 1, 1], width_ratios=[1, 0.025], wspace=0.05)

# Define subplot, add panel title and label
def plot_land():
    ax.set_title(panel_name[ii], loc='left', fontweight='bold')    
    ax.set_xlim([-280, 80])
    ax.set_ylim([-78.7, -59.8])
    # Add land 
    ax.contourf(ht.xt_ocean, land_mask_lat, land_mask_masked, colors='darkgrey', zorder=2)
    ax.contour(contour_xt_ocean, contour_yt_ocean, contour_1000m, [1000],colors='k', linewidths=0.5)

for ii in range(4):
    if ii == 0:
        # Age, WIND+
        ax = fig.add_subplot(gs[ii])
        ax.set_title(title_name[ii])
        plot_land()
        cf = ax.pcolormesh(bottom_age_con.xt_ocean, bottom_age_con.yt_ocean, (bottom_age_up-bottom_age_con), cmap=cmocean.balance, vmin=-15, vmax=15) 
        # Masked DSW regions
        ax.add_patch(poly1)
        ax.add_patch(poly2)
        ax.add_patch(poly3)
        ax.add_patch(poly4)
    elif ii == 1:
        # Age, WIND-
        ax = fig.add_subplot(gs[ii+1])
        ax.set_title(title_name[ii])
        plot_land()
        cf = ax.pcolormesh(bottom_age_con.xt_ocean, bottom_age_con.yt_ocean, (bottom_age_down-bottom_age_con), cmap=cmocean.balance, vmin=-15, vmax=15) 
    elif ii == 2:
        # Salt, WIND+
        ax = fig.add_subplot(gs[ii+2])
        ax.set_title(title_name[ii])
        plot_land()
        CF = ax.pcolormesh(bottom_salt_con.xt_ocean, bottom_salt_con.yt_ocean, (bottom_salt_up-bottom_salt_con), cmap=cmocean.delta, vmin=-0.06, vmax=0.06) 
    elif ii == 3:
        # Salt, WIND-
        ax = fig.add_subplot(gs[ii+3])
        ax.set_title(title_name[ii])
        plot_land()
        CF = ax.pcolormesh(bottom_salt_con.xt_ocean, bottom_salt_con.yt_ocean, (bottom_salt_down-bottom_salt_con), cmap=cmocean.delta, vmin=-0.06, vmax=0.06) 
        plt.xlabel('Longitude ($^{\circ}$E)')
    plt.ylabel('Latitude ($^{\circ}$N)')
    if ii < 3:
        plt.tick_params(labelbottom=False)
        
        
# Add colorbars
cbar_ax = fig.add_subplot(gs[0:2, -1])
cbar = plt.colorbar(cf, cax=cbar_ax, fraction=0.03, pad=0.01, extend='both')
cbar.set_label('Bottom age anomaly (years)')
cbar_ax = fig.add_subplot(gs[2:, -1])
cbar = plt.colorbar(CF, cax=cbar_ax, fraction=0.03, pad=0.01, extend='both')
cbar.set_label('Bottom salinity anomaly')

# Finished
save_fig = 1
if save_fig == 1:
    fig_name = 'Figure3.png'
    plt.savefig(fig_path + fig_name)